In [10]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + '/../src')


In [11]:
import logging
import wandb
from concurrent.futures import ThreadPoolExecutor
import multiprocessing


In [12]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('download.log')
    ]
)
logger = logging.getLogger(__name__)

# Set multiprocessing start method to 'spawn'
multiprocessing.set_start_method("spawn", force=True)

In [13]:
# Configuration
download_dir = "/scratch/downloaded_artifacts"
entity = "wlp9800-new-york-university"
project_name = "oho_exps"
group_name = "batched_bptt_mnist-fixed-1_7b11a6cf504f46bc93a041cf1ffcf207"
max_download_workers = 20

In [14]:
# Ensure download directory exists
os.makedirs(download_dir, exist_ok=True)

In [15]:
# Function to download a single run's artifact
def download_artifact(run_data):
    run_id = run_data["id"]
    config = run_data["config"]
    try:
        api = wandb.Api()
        artifact = api.artifact(f'{entity}/{project_name}/logs_{run_id}:v0')
        artifact_dir = os.path.join(download_dir, artifact.name)
        artifact.download(root=artifact_dir)
        logger.info(f"Downloaded {artifact.name} to {artifact_dir}")
        return {
            "run_id": run_id,
            "artifact_dir": artifact_dir,
            "config": config,
            "summary": run_data["summary"],
            "status": "success"
        }
    except Exception as e:
        logger.error(f"Error downloading artifact for run {run_id}: {str(e)}")
        return {
            "run_id": run_id,
            "artifact_dir": None,
            "config": config,
            "status": f"error: {str(e)}"
        }

In [16]:
api = wandb.Api()
runs = api.runs(
    path=f"{entity}/{project_name}",
    filters={"group": group_name}
)

# Prepare run data
run_data = [{"id": run.id, "config": run.config} for run in runs]
logger.info(f"Found {len(run_data)} runs to download")


2025-04-30 16:31:03,603 - INFO - Found 100 runs to download


In [17]:
with ThreadPoolExecutor(max_workers=max_download_workers) as executor:
    download_results = list(executor.map(download_artifact, run_data))

2025-04-30 16:31:03,926 - ERROR - Error downloading artifact for run re8zu2b3: HTTP 404: failed to find artifact collection logs_re8zu2b3


wandb: Downloading large artifact logs_akta8vmn:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_3dm43avs:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_0qoyxlle:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_n95bo8vu:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_ft1lxip6:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_beg2r84h:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_ljkww0uu:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_p3nm17w9:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_jwm36ml2:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_d9caj7ve:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_qw142rem:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_tt2z1d30:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_wz2ln8y6:v0, 58.75MB. 1 files... 
wandb: Downloading large artifact logs_qc7teymp:v0,

2025-04-30 16:31:05,733 - INFO - Downloaded logs_p3nm17w9:v0 to /scratch/downloaded_artifacts/logs_p3nm17w9:v0
2025-04-30 16:31:05,734 - ERROR - Error downloading artifact for run p3nm17w9: 'summary'


wandb:   1 of 1 files downloaded.  
Done. 0:0:1.6


2025-04-30 16:31:05,802 - INFO - Downloaded logs_avctuqxw:v0 to /scratch/downloaded_artifacts/logs_avctuqxw:v0


...

2025-04-30 16:31:14,648 - INFO - Downloaded logs_nzmzzf0p:v0 to /scratch/downloaded_artifacts/logs_nzmzzf0p:v0
2025-04-30 16:31:14,648 - ERROR - Error downloading artifact for run nzmzzf0p: 'summary'


wandb:   1 of 1 files downloaded.  
Done. 0:0:0.7


2025-04-30 16:31:14,668 - INFO - Downloaded logs_w0m5e60i:v0 to /scratch/downloaded_artifacts/logs_w0m5e60i:v0
2025-04-30 16:31:14,668 - ERROR - Error downloading artifact for run w0m5e60i: 'summary'


wandb: Downloading large artifact logs_py1cdgbz:v0, 58.75MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.5


2025-04-30 16:31:16,500 - INFO - Downloaded logs_py1cdgbz:v0 to /scratch/downloaded_artifacts/logs_py1cdgbz:v0
2025-04-30 16:31:16,500 - ERROR - Error downloading artifact for run py1cdgbz: 'summary'


In [18]:
# Save download results for the processing script, including group name
import pickle
results_file = os.path.join(download_dir, f'download_results_{group_name}.pkl')
with open(results_file, 'wb') as f:
    pickle.dump(download_results, f)
logger.info(f"Saved download results to {results_file}")

2025-04-30 16:31:16,512 - INFO - Saved download results to /scratch/downloaded_artifacts/download_results_batched_bptt_mnist-fixed-1_7b11a6cf504f46bc93a041cf1ffcf207.pkl
